<a href="https://colab.research.google.com/github/Ashvitharavichandran/CI-CD/blob/main/MBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#library
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
#Load Dataset
df = pd.read_csv("Groceries_dataset.csv")
print(df.shape)

(38765, 3)


In [3]:
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [4]:
# STEP 2: DATA CLEANING
# Convert date
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
df["Date"]


,Date
0,2015-07-21
1,2015-01-05
2,2015-09-19
3,2015-12-12
4,2015-02-01
...,...
38760,2014-10-08
38761,2014-02-23
38762,2014-04-16
38763,2014-12-03


In [5]:
# Clean itemDescription to avoid duplicate products
# - strip spaces
# - convert to lowercase
# - remove commas
df["itemDescription"] = (
    df["itemDescription"]
    .str.strip()
    .str.lower()
    .str.replace(",", "", regex=False)
)

In [6]:
print(df.shape)

(38765, 3)


In [7]:
# STEP 3: CREATE TRANSACTION ID
# Since there is NO invoice ID in this dataset,


df["Transaction_ID"] = (
    df["Member_number"].astype(str) + "_" +
    df["Date"].dt.strftime("%Y-%m-%d")
)

# Each Transaction_ID now represents one shopping basket


In [8]:
# STEP 4: CUSTOMER SEGMENTATION USING KMEANS

# Aggregate item-level data to customer-level behavior
customer_features = df.groupby("Member_number").agg(
    Num_Transactions=("Transaction_ID", "nunique"),  # how often customer shops
    Total_Items=("itemDescription", "count")          # how many items customer buys
)

customer_features.head()

,Num_Transactions,Total_Items
Member_number,,
1000,5,13
1001,5,12
1002,4,8
1003,4,8
1004,8,21


In [9]:
# 4.2 FEATURE SCALING (MANDATORY FOR KMEANS)

# KMeans is distance-based → features must be on same scale
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features)

In [10]:
#4.3 APPLY KMEANS CLUSTERING

# Create KMeans model with 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)



In [11]:
# Assign each customer to a cluster
customer_features["Segment"] = kmeans.fit_predict(scaled_features)

In [12]:
# 4.4 INTERPRET CLUSTERS USING CLUSTER CENTERS

# Convert cluster centers back to original scale
cluster_centers = pd.DataFrame(
    scaler.inverse_transform(kmeans.cluster_centers_),
    columns=customer_features.columns[:2]
)


In [13]:
# Add cluster id
cluster_centers["Cluster"] = cluster_centers.index

cluster_centers

,Num_Transactions,Total_Items,Cluster
0,4.454201,11.487682,0
1,2.192666,5.423749,1
2,6.943049,18.864322,2


In [14]:
# 4.5 MAP CLUSTERS TO MEANINGFUL SEGMENT NAMES

# Initialize all as Regular Buyers
cluster_centers["Segment_Name"] = "Regular Buyers"

# Cluster with highest transactions → Frequent Buyers
cluster_centers.loc[
    cluster_centers["Num_Transactions"].idxmax(),
    "Segment_Name"
] = "Frequent Buyers"

# Cluster with lowest transactions → Occasional Buyers
cluster_centers.loc[
    cluster_centers["Num_Transactions"].idxmin(),
    "Segment_Name"
] = "Occasional Buyers"


In [15]:
# Create mapping from cluster number to segment name
segment_map = cluster_centers.set_index("Cluster")["Segment_Name"].to_dict()
segment_map

{0: 'Regular Buyers', 1: 'Occasional Buyers', 2: 'Frequent Buyers'}

In [16]:
# Assign readable segment names to customers
customer_features["Segment_Name"] = customer_features["Segment"].map(segment_map)
customer_features

,Num_Transactions,Total_Items,Segment,Segment_Name
Member_number,,,,
1000,5,13,0,Regular Buyers
1001,5,12,0,Regular Buyers
1002,4,8,0,Regular Buyers
1003,4,8,0,Regular Buyers
1004,8,21,2,Frequent Buyers
...,...,...,...,...
4996,3,10,0,Regular Buyers
4997,2,6,1,Occasional Buyers
4998,1,2,1,Occasional Buyers


STEP 5: MARKET BASKET ANALYSIS

In [17]:
# 5.1 CREATE BASKET MATRIX
# Rows → Transaction_ID (basket)
# Columns → itemDescription (products)
# Values → count of item in basket
basket = (
    df.groupby(["Transaction_ID", "itemDescription"])
    .size()
    .unstack(fill_value=0)
)

# Convert counts to binary (1 = item present, 0 = absent)
basket = (basket > 0).astype(int)
basket

itemDescription,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,...,uht-milk,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
Transaction_ID,,,,,,,,,,,,,,,,,,,,,
1000_2014-06-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1000_2015-03-15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1000_2015-05-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000_2015-07-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000_2015-11-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999_2015-05-16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4999_2015-12-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5000_2014-03-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# 5.2 RUN APRIORI ALGORITHM
# Find frequent itemsets with minimum 2% support
frequent_itemsets = apriori(
    basket,
    min_support=0.001,
    use_colnames=True
)

In [46]:
# 5.3 GENERATE ASSOCIATION RULES

rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.2
)

rules = rules[rules["lift"] >= 1.05]

rules.sort_values("lift", ascending=False).head()



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1,"(sausage, yogurt)",(whole milk),0.005748,0.157923,0.001470,0.255814,1.619866,1.0,0.000563,1.131541,0.384877,0.009065,0.116250,0.132562
0,"(rolls/buns, sausage)",(whole milk),0.005347,0.157923,0.001136,0.212500,1.345594,1.0,0.000292,1.069304,0.258214,0.007007,0.064813,0.109847


STEP 6: SEGMENT-WISE MARKET BASKET ANALYSIS

In [36]:
# 6.1 MERGE SEGMENT INFO BACK TO TRANSACTIONS
df_seg = df.merge(
    customer_features[["Segment_Name"]],
    left_on="Member_number",
    right_index=True
)
df_seg.head()


,Member_number,Date,itemDescription,Transaction_ID,Segment_Name
0,1808,2015-07-21,tropical fruit,1808_2015-07-21,Regular Buyers
1,2552,2015-01-05,whole milk,2552_2015-01-05,Regular Buyers
2,2300,2015-09-19,pip fruit,2300_2015-09-19,Regular Buyers
3,1187,2015-12-12,other vegetables,1187_2015-12-12,Occasional Buyers
4,3037,2015-02-01,whole milk,3037_2015-02-01,Occasional Buyers


In [47]:
# 6.2 FUNCTION TO RUN MBA FOR A GIVEN SEGMENT

def mba_by_segment(segment_name):
    # Filter transactions for the given segment
    temp = df_seg[df_seg["Segment_Name"] == segment_name]

    if temp.empty:
        return pd.DataFrame()

    # Create basket
    basket = (
        temp.groupby(["Transaction_ID", "itemDescription"])
        .size()
        .unstack(fill_value=0)
    )

    basket = (basket > 0).astype(int)

    # LOWER support for segment-wise MBA
    freq = apriori(
        basket,
        min_support=0.002,   # 🔥 critical fix
        use_colnames=True
    )

    # Ensure item pairs exist
    freq["length"] = freq["itemsets"].apply(len)
    if not (freq["length"] >= 2).any():
        return pd.DataFrame()

    # Generate rules using CONFIDENCE
    rules = association_rules(
        freq,
        metric="confidence",
        min_threshold=0.15
    )

    # Light lift filtering
    rules = rules[rules["lift"] >= 1.01]

    return rules.sort_values("lift", ascending=False)




In [50]:
frequent_rules = mba_by_segment("Frequent Buyers")
regular_rules = mba_by_segment("Regular Buyers")
occasional_rules = mba_by_segment("Occasional Buyers")

frequent_rules.head()
regular_rules.head()
occasional_rules.head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
9,(processed cheese),(root vegetables),0.009822,0.066100,0.002124,0.216216,3.271030,1.0,0.001474,1.191527,0.701173,0.028777,0.160741,0.124172
2,(frozen vegetables),(root vegetables),0.023892,0.066100,0.003716,0.155556,2.353324,1.0,0.002137,1.105934,0.589145,0.043077,0.095787,0.105890
3,(fruit/vegetable juice),(rolls/buns),0.031325,0.109902,0.005840,0.186441,1.696430,1.0,0.002398,1.094079,0.423802,0.043137,0.085989,0.119790
1,(candy),(rolls/buns),0.011680,0.109902,0.002124,0.181818,1.654370,1.0,0.000840,1.087898,0.400215,0.017778,0.080796,0.100571
4,(grapes),(other vegetables),0.014600,0.116804,0.002655,0.181818,1.556612,1.0,0.000949,1.079462,0.362877,0.020619,0.073613,0.102273


In [39]:
# ------------------------------------------------------------
# 6.3 RUN SEGMENT-WISE MBA
# ------------------------------------------------------------

frequent_rules = mba_by_segment("Frequent Buyers")

frequent_rules

regular_rules = mba_by_segment("Regular Buyers")
occasional_rules = mba_by_segment("Occasional Buyers")


In [51]:
# STEP 7: CROSS-SELL RECOMMENDATIONS
recommendations = frequent_rules[
    (frequent_rules["confidence"] >= 0.3) &
    (frequent_rules["lift"] >= 1.1) &
    (frequent_rules["support"] >= 0.005)
][["antecedents", "consequents", "confidence", "lift"]]



In [24]:
recommendations = frequent_rules[
    (frequent_rules["confidence"] > 0.1) &
    (frequent_rules["lift"] > 1.4) &
    (frequent_rules["antecedents"].apply(lambda x: len(x) == 1))
][["antecedents", "consequents", "confidence", "lift"]]

recommendations.head(2)


,antecedents,consequents,confidence,lift
7,(chocolate),(rolls/buns),0.185841,1.685579
10,(frankfurter),(rolls/buns),0.159509,1.446752


In [ ]:
# Export customer segmentation
customer_features.to_csv("customer_segments_kmeans.csv")


Confidence > 0.4: 7
Lift > 1.5: 0


In [ ]:
# Export cross-sell recommendations
recommendations.to_csv("cross_sell_recommendations_kmeans.csv")
